In [6]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName('less_pysqprk_DF').\
        master('local').\
        config('spark.executor.memory', "512m").\
        config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000").\
        config("spark.hadoop.fs.default.name", "hdfs://localhost:9000").\
        getOrCreate()

/home/ilyagasan/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:
#spark.stop()

In [7]:
spark

In [18]:
df = spark.read.csv('hdfs://localhost:9000/data/Sales_April_2019.csv', header=True, inferSchema=True)

In [19]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95|04/19/19 08:46|917 1st St, Dalla...|
|    null|                null|            null|      null|          null|                null|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/19 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|     600.0|04/12/19 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|04/12/19 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99|04/30/19 09:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|04/29/19 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|         

In [24]:
df.schema

StructType([StructField('Order ID', IntegerType(), True), StructField('Product', StringType(), True), StructField('Quantity Ordered', IntegerType(), True), StructField('Price Each', DoubleType(), True), StructField('Order Date', StringType(), True), StructField('Purchase Address', StringType(), True)])

In [26]:
df.count()

18383

In [27]:
df = df.drop_duplicates()

In [28]:
df.count()

18269

In [30]:
from pyspark.sql.functions import lit

df_none = df.select([lit(None) for i in df.columns])

In [31]:
df_none.show()

+----+----+----+----+----+----+
|NULL|NULL|NULL|NULL|NULL|NULL|
+----+----+----+----+----+----+
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
|null|null|null|null|null|null|
+----+----+----+----+----+----+
only showing top 20 rows



In [32]:
df = df.union(df_none)

In [33]:
df.count()

36538

In [34]:
df = df.na.drop()

In [35]:
df.count()

18267

In [38]:
from pyspark.sql.functions import col, ceil

In [43]:
df = df.withColumn('unit_price_profit', col("Quantity Ordered")* col("Price Each"))

In [44]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+-----------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|unit_price_profit|
+--------+--------------------+----------------+----------+--------------+--------------------+-----------------+
|  176675|AAA Batteries (4-...|               1|      2.99|04/02/19 09:02|933 Meadow St, Se...|             2.99|
|  176725|AA Batteries (4-p...|               2|      3.84|04/15/19 14:33|630 14th St, San ...|             7.68|
|  176925|Lightning Chargin...|               1|     14.95|04/18/19 15:31|920 Wilson St, Ne...|            14.95|
|  176991|Bose SoundSport H...|               1|     99.99|04/15/19 01:49|338 2nd St, San F...|            99.99|
|  177141|AAA Batteries (4-...|               1|      2.99|04/20/19 16:12|559 4th St, San F...|             2.99|
|  177177|AA Batteries (4-p...|               1|      3.84|04/08/19 12:10|182 12th St, S

In [47]:
df_sum = df.groupby('Quantity Ordered' ).sum('unit_price_profit')

In [48]:
df_sum.show()

+----------------+----------------------+
|Quantity Ordered|sum(unit_price_profit)|
+----------------+----------------------+
|               1|     3350854.210000706|
|               6|    179.33999999999997|
|               3|     4064.429999999983|
|               5|    392.79999999999984|
|               4|    1045.0000000000007|
|               7|                 20.93|
|               2|     38050.14000000027|
+----------------+----------------------+



In [49]:
df_avg = df.groupby('Quantity Ordered').avg('unit_price_profit')

In [50]:
df_avg.show()

+----------------+----------------------+
|Quantity Ordered|avg(unit_price_profit)|
+----------------+----------------------+
|               1|    202.63994980652552|
|               6|    25.619999999999997|
|               3|    13.593411371237401|
|               5|     16.36666666666666|
|               4|    14.513888888888898|
|               7|                 20.93|
|               2|    28.652213855421888|
+----------------+----------------------+



In [51]:
join_df = df_sum.join(df_avg, 'Quantity Ordered', 'left')

In [52]:
join_df.show()

+----------------+----------------------+----------------------+
|Quantity Ordered|sum(unit_price_profit)|avg(unit_price_profit)|
+----------------+----------------------+----------------------+
|               1|     3350854.210000706|    202.63994980652552|
|               6|    179.33999999999997|    25.619999999999997|
|               3|     4064.429999999983|    13.593411371237401|
|               5|    392.79999999999984|     16.36666666666666|
|               4|    1045.0000000000007|    14.513888888888898|
|               7|                 20.93|                 20.93|
|               2|     38050.14000000027|    28.652213855421888|
+----------------+----------------------+----------------------+



In [60]:
df.rdd.getNumPartitions()

2

In [55]:
join_df.rdd.getNumPartitions()

1

In [59]:
df = df.coalesce(2)

In [63]:
df.write.format('csv').partitionBy('Product').option('header', True).save('hdfs://localhost:9000 ')

In [1]:
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

!java -version


openjdk version "1.8.0_422"
OpenJDK Runtime Environment (build 1.8.0_422-8u422-b05-1~24.04-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)
